In [125]:
from koeda import EDA, RD, RI, SR, RS
import pandas as pd
from tqdm import tqdm
import random

In [ ]:
from KorEDA import eda

c:\Users\bestm\Desktop\Dev\korean-text-augmentation\KorEDA\eda.py:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [ ]:
df = pd.read_csv('./dataset/train.csv')

In [ ]:
augmenter = EDA(morpheme_analyzer="Okt")

In [ ]:
num_aug = 4

In [ ]:
augmented_data = {"id": [], "text": [], "label": []}

for i, row in tqdm(df.iterrows(), total=len(df)):
    id = row["id"]
    label = row['label']
    text = row['text']
    
    # To prevent dupcliates, first augment 30 times and then sample 4 of them
    augmented_text = augmenter(text, repetition=30)
    augmented_text = list(set(augmented_text))
    #random sample
    augmented_text = random.sample(augmented_text, num_aug)
    
    assert len(augmented_text) <= num_aug

    augmented_data["id"].extend([id] * (num_aug + 1))
    augmented_data["text"].extend([text] + augmented_text)
    augmented_data["label"].extend([label] * (num_aug + 1))

100%|██████████| 2000/2000 [06:47<00:00,  4.91it/s]


In [ ]:
len(augmented_data["text"])

10000

In [ ]:
augmented_df = pd.DataFrame(augmented_data)

In [ ]:
augmented_df['label'].value_counts()

1    5000
0    5000
Name: label, dtype: int64

In [ ]:
augmented_df.to_csv("./dataset/train_aug_eda.csv", index=False)
augmented_df.drop(columns=["id"]).to_csv("./dataset/train_aug_eda_noid.csv", index=False)

# Postprocessing - Backtranslation

In [338]:
train_df = pd.read_csv('./dataset/train.csv')
train_df

,id,text,label
0,218,가성비 굿이에요 저는 프레임만 구입했는데 2주만에 왔고요 디자인 깔끔하고요 인터넷 ...,1
1,338,장갑이 짱짱하고 패드도 좋아요. 장갑낀 상태에서 스마트폰 터치도 됩니다.,1
2,736,만족 대만족 주파수만 맞춰서 연결하면 불루투스 오디오 내장안되어있어도 이렇게 좋습니...,1
3,955,고기가 맛있다고 하신분들...믿고 구매했는데... 저울달아보니 그램수도 안맞고 고기...,0
4,1005,기간도일주일이나되어도기다렸는데 한상자는좋은데한상자는블루베리가작고크기가다다르다고하네요...,0
...,...,...,...
1995,199505,"아들을 위해 구입한 책상입니다. 책상 좋습니다. 초등학생, 중학생 책상으로 딱 입니...",1
1996,199559,김장김치가 너무 익어서 새 김치가 드시고 싶다 하셔서 부모님께 보내드렸어요 좋다고 ...,1
1997,199577,재구매 아이 레고정리용으로 사용하고있어요 레고가 점점 늘어나서 또 구매했어요 정리하...,1
1998,199601,완전좋아요 아이들이너무좋아합니다 또구매하고싶네요 또구매할께오,1


In [328]:
origin_df = pd.read_csv('./dataset/train_aug_backtranslate_origin.csv')
origin_df

,id,text,label,en,jp,zh-CN,fr,es
0,218,가성비 굿이에요 저는 프레임만 구입했는데 2주만에 왔고요 디자인 깔끔하고요 인터넷 ...,1,좋은 가격입니다. 나는 프레임 만 샀지 만 2 주 만에왔다. 디자인은 깨끗합니다.,좋은 가격입니다. 나는 프레임 만 샀지 만 2 주 만에왔다. 디자인은 아름답습니다.,이것은 좋은 가격입니다. 나는 프레임 워크 만 구입했지만 2 주 안에왔다. 디자인은...,좋은 가격입니다. 나는 단지 설정을 샀지 만 2 주 만에왔다. 디자인은 깨끗합니다.,좋은 가격입니다. 프레임 만 샀지 만 2 주 안에 도착했습니다. 디자인은 깨끗합니다.
1,338,장갑이 짱짱하고 패드도 좋아요. 장갑낀 상태에서 스마트폰 터치도 됩니다.,1,장갑은 훌륭하고 패드는 좋습니다. 장갑 동안 스마트 폰을 만질 수도 있습니다.,장갑은 훌륭하고 패드는 좋습니다. 장갑 중에 스마트 폰을 만질 수도 있습니다.,장갑은 훌륭하고 쿠션이 좋습니다. 장갑 중에 스마트 폰을 만질 수도 있습니다.,장갑은 훌륭하고 패드는 좋습니다. 장갑 중에 스마트 폰을 만질 수도 있습니다.,장갑은 훌륭하고 패드는 좋습니다. 장갑을 끼며 스마트 폰을 만질 수도 있습니다.
2,736,만족 대만족 주파수만 맞춰서 연결하면 불루투스 오디오 내장안되어있어도 이렇게 좋습니...,1,Bulutus 오디오에 구축 된 내용이 없어도 주파수 만족도 만 연결하면 검은 색도...,주파수 만족도 만 연결하는 경우 Bulutus의 오디오에 내장되어 있지 않더라도 검...,만족 주파수 만 연결하는 경우 Bulutus 오디오에 구축 된 내용이 없더라도 Bl...,빈도 만족도 만 연결하면 Bulutus 오디오가 만들어지지 않더라도 검은 색도 우아...,주파수의 만족 만 연결되면 Bulutus의 오디오가 제작되지 않더라도 검은 색은 우...
3,955,고기가 맛있다고 하신분들...믿고 구매했는데... 저울달아보니 그램수도 안맞고 고기...,0,고기가 맛있다고 말한 사람들은 ... 나는 믿고 그것을 샀다 ... 나는 그램을 살...,고기가 맛있다고 말한 사람들은 ... 나는 믿고 샀다 ... 나는 그램을 살 수 있...,고기가 맛있다고 말하는 사람들은 ... 나는 믿고 살 수 있습니다 ... 나는 그램...,고기가 맛있다고 말한 사람들 ... 나는 생각하고 그것을 샀다 ... 나는 그램을 ...,고기가 맛있다고 말한 사람들은 ... 나는 그것을 믿고 그것을 샀다 ... 나는 그...
4,1005,기간도일주일이나되어도기다렸는데 한상자는좋은데한상자는블루베리가작고크기가다다르다고하네요...,0,나는 일주일을 기다리고 있었지만 상자는 좋다.,나는 그 주를 기다리고 있었지만 상자는 좋다.,나는 일주일을 기다리고 있었지만 상자는 매우 좋습니다.,나는 일주일을 기다리고 있었지만 상자는 좋다.,나는 일주일을 기다리고 있었지만 상자는 좋다.
...,...,...,...,...,...,...,...,...
1995,199505,"아들을 위해 구입한 책상입니다. 책상 좋습니다. 초등학생, 중학생 책상으로 딱 입니...",1,"이것은 아들을 위해 구입 한 책상입니다. 좋은 책상. 초등학생, 중학생 책상에 적합...",이것은 내 아들을 위해 구입 한 책상입니다. 좋은 책상. 초등 및 중학교 학생들을위...,"이것은 내 아들을위한 테이블입니다. 좋은 책상. 초등학생, 중학생의 책상에 매우 적...",이것은 아들을 위해 구입 한 사무실입니다. 좋은 사무실. 중학생의 사무실 인 초등학...,"이것은 아이를 위해 구입 한 책상입니다. 좋은 책상. 초등학생, 고등학생에게 적합합..."
1996,199559,김장김치가 너무 익어서 새 김치가 드시고 싶다 하셔서 부모님께 보내드렸어요 좋다고 ...,1,Kimjang Kimchi는 너무 요리되어 새로운 Kimchi를 먹고 싶었 기 때문...,Kim Jean Kimchi는 너무 요리되어 새로운 김치를 먹고 싶었 기 때문에 부...,Kimjang Kimchi는 너무 요리되어 새로운 Kimchi를 먹고 싶기 때문에 ...,Kimjang Kimchi는 너무 요리되어 새로운 Kimchi를 먹고 싶었 기 때문...,Kimjang Kimchi는 너무 요리되어 새로운 Kimchi를 먹고 싶었 기 때문...
1997,199577,재구매 아이 레고정리용으로 사용하고있어요 레고가 점점 늘어나서 또 구매했어요 정리하...,1,나는 Irego의 재생을 위해 그것을 사용하고 있습니다. 레고가 증가하고 다시 구입...,나는 그것을 사용하여 iirego를 연주합니다. 레고가 증가하고 다시 구입했습니다.,IREGO의 재생을 위해 사용하고 있습니다. 레고가 증가했고 다시 구입했습니다.,나는 Irego의 재생에 그것을 사용합니다. 레고가 증가하고 다시 구입했습니다.,IREGO의 재생을 위해 사용하고 있습니다. 레고는 증가하여 다시 구입했습니다.
1998,199601,완전좋아요 아이들이너무좋아합니다 또구매하고싶네요 또구매할께오,1,난 정말 좋아. 나는 그것을 너무 좋아한다. 다시 사고 싶어요.,난 정말 좋아. 난 정말 좋아. 다시 사고 싶어요.,난 정말 좋아. 나는 그것을 아주 많이 좋아. 다시 사고 싶어요.,나는 정말 좋아한다. 나는 그를 너무 사랑합니다. 다시 사고 싶어요.,난 정말 좋아. 나는 그것을 많이 좋아. 다시 사고 싶어요.


In [344]:
# Convert each translation columns to single row
aug_column = [c for c in origin_df.columns if c not in ['id', 'text', 'label']]
rows = []
for i, row in origin_df.iterrows():
    id = row["id"]
    label = row['label']
    text = row['text']
    for c in aug_column:
        rows.append({'id': f'{id}a', 'text': row[c], 'label': label})

aug_df = pd.DataFrame(rows)

In [345]:
aug_df

,id,text,label
0,218a,좋은 가격입니다. 나는 프레임 만 샀지 만 2 주 만에왔다. 디자인은 깨끗합니다.,1
1,218a,좋은 가격입니다. 나는 프레임 만 샀지 만 2 주 만에왔다. 디자인은 아름답습니다.,1
2,218a,이것은 좋은 가격입니다. 나는 프레임 워크 만 구입했지만 2 주 안에왔다. 디자인은...,1
3,218a,좋은 가격입니다. 나는 단지 설정을 샀지 만 2 주 만에왔다. 디자인은 깨끗합니다.,1
4,218a,좋은 가격입니다. 프레임 만 샀지 만 2 주 안에 도착했습니다. 디자인은 깨끗합니다.,1
...,...,...,...
9995,199719a,예쁘다 .. 잔잔하고있다.,1
9996,199719a,아름답습니다 .. 그것은 끈기가 있습니다.,1
9997,199719a,매우 아름답습니다 .. 매우 NA.,1
9998,199719a,예쁘다 .. 괴롭힘입니다.,1


In [346]:
# Drop duplicate texts
aug_df.drop_duplicates(subset=['text'], inplace=True)
aug_df

,id,text,label
0,218a,좋은 가격입니다. 나는 프레임 만 샀지 만 2 주 만에왔다. 디자인은 깨끗합니다.,1
1,218a,좋은 가격입니다. 나는 프레임 만 샀지 만 2 주 만에왔다. 디자인은 아름답습니다.,1
2,218a,이것은 좋은 가격입니다. 나는 프레임 워크 만 구입했지만 2 주 안에왔다. 디자인은...,1
3,218a,좋은 가격입니다. 나는 단지 설정을 샀지 만 2 주 만에왔다. 디자인은 깨끗합니다.,1
4,218a,좋은 가격입니다. 프레임 만 샀지 만 2 주 안에 도착했습니다. 디자인은 깨끗합니다.,1
...,...,...,...
9995,199719a,예쁘다 .. 잔잔하고있다.,1
9996,199719a,아름답습니다 .. 그것은 끈기가 있습니다.,1
9997,199719a,매우 아름답습니다 .. 매우 NA.,1
9998,199719a,예쁘다 .. 괴롭힘입니다.,1


In [343]:
train_df['id'] = train_df['id'].astype(str)
train_df

,id,text,label
0,218,가성비 굿이에요 저는 프레임만 구입했는데 2주만에 왔고요 디자인 깔끔하고요 인터넷 ...,1
1,338,장갑이 짱짱하고 패드도 좋아요. 장갑낀 상태에서 스마트폰 터치도 됩니다.,1
2,736,만족 대만족 주파수만 맞춰서 연결하면 불루투스 오디오 내장안되어있어도 이렇게 좋습니...,1
3,955,고기가 맛있다고 하신분들...믿고 구매했는데... 저울달아보니 그램수도 안맞고 고기...,0
4,1005,기간도일주일이나되어도기다렸는데 한상자는좋은데한상자는블루베리가작고크기가다다르다고하네요...,0
...,...,...,...
1995,199505,"아들을 위해 구입한 책상입니다. 책상 좋습니다. 초등학생, 중학생 책상으로 딱 입니...",1
1996,199559,김장김치가 너무 익어서 새 김치가 드시고 싶다 하셔서 부모님께 보내드렸어요 좋다고 ...,1
1997,199577,재구매 아이 레고정리용으로 사용하고있어요 레고가 점점 늘어나서 또 구매했어요 정리하...,1
1998,199601,완전좋아요 아이들이너무좋아합니다 또구매하고싶네요 또구매할께오,1


In [347]:
# Sample 10,000 rows
aug_df = aug_df.sample(n=8000)
aug_df = pd.concat([train_df, aug_df]).sort_values(by=['id']).reset_index(drop=True)
aug_df

,id,text,label
0,100037,맛있어요 어버이날에 부모님께 가져다 드릴려고 사고 저희가 먼저 맛보려고 샀어요 생각...,1
1,100037a,이 맛있어요. 나는 어머니의 날에 부모님에게 가져 오기 위해 그것을 샀다. 우리는 ...,1
2,100037a,맛있어요. 나는 어머니의 날에 부모님에게 가져 오기 위해 그것을 샀다. 우리는 그것...,1
3,100037a,이 맛있어요. 나는 어머니의 날에 부모님에게 가져 가기 위해 그것을 샀다. 우리는 ...,1
4,100190,저렴하게 구매했습니다^^ 포장도 신경써 주셔서 감사합니다~많이 파세요^^,1
...,...,...,...
9995,99979,"다리가 비뚤어진 것도있고 등 폭도 좁고, 예전에 구매했던것과 똑같은걸 산다고 샀는데...",0
9996,99979a,다리가 구부러지고 등이 좁습니다. 이전에 구입 한 것과 같은 것을 샀지 만 제품은 ...,0
9997,99979a,다리가 구부러졌고 등이 좁았고 이전에 구입 한 것과 같은 것을 샀지 만 제품은 약간...,0
9998,99979a,다리가 뒤틀리고 등이 좁고 이전에 구입 한 것과 같은 것을 샀지 만 제품은 약간 불...,0


In [348]:
aug_df['id'] = aug_df['id'].apply(lambda x: x.replace('a', ''))
aug_df

,id,text,label
0,100037,맛있어요 어버이날에 부모님께 가져다 드릴려고 사고 저희가 먼저 맛보려고 샀어요 생각...,1
1,100037,이 맛있어요. 나는 어머니의 날에 부모님에게 가져 오기 위해 그것을 샀다. 우리는 ...,1
2,100037,맛있어요. 나는 어머니의 날에 부모님에게 가져 오기 위해 그것을 샀다. 우리는 그것...,1
3,100037,이 맛있어요. 나는 어머니의 날에 부모님에게 가져 가기 위해 그것을 샀다. 우리는 ...,1
4,100190,저렴하게 구매했습니다^^ 포장도 신경써 주셔서 감사합니다~많이 파세요^^,1
...,...,...,...
9995,99979,"다리가 비뚤어진 것도있고 등 폭도 좁고, 예전에 구매했던것과 똑같은걸 산다고 샀는데...",0
9996,99979,다리가 구부러지고 등이 좁습니다. 이전에 구입 한 것과 같은 것을 샀지 만 제품은 ...,0
9997,99979,다리가 구부러졌고 등이 좁았고 이전에 구입 한 것과 같은 것을 샀지 만 제품은 약간...,0
9998,99979,다리가 뒤틀리고 등이 좁고 이전에 구입 한 것과 같은 것을 샀지 만 제품은 약간 불...,0


In [349]:
aug_df.to_csv("./dataset/train_aug_backtranslate.csv", index=False)